# Basic Custom Studio Container

This notebook demonstrates how to build basic custom Docker container to be used as Amazon SageMaker Studio Kernel Gateway application. Reference documentation is available at https://docs.aws.amazon.com/sagemaker/latest/dg/studio-byoi-create-sdk.html

We start by defining some variables like the current execution role, the ECR repository that we are going to use for pushing the custom Docker container.

In [1]:
import boto3
import sagemaker
from sagemaker import get_execution_role

ecr_namespace = 'sagemaker-studio-containers/'
prefix = 'basic-studio-container'

ecr_repository_name = ecr_namespace + prefix
role = get_execution_role()
account_id = role.split(':')[4]
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
bucket = sagemaker_session.default_bucket()

print(account_id)
print(region)
print(role)
print(bucket)

825935527263
eu-west-1
arn:aws:iam::825935527263:role/service-role/AmazonSageMaker-ExecutionRole-endtoendml
sagemaker-eu-west-1-825935527263


## Create the container

Let's take a look at the Dockerfile which defines the statements for building our custom SageMaker Studio container:

In [106]:
! pygmentize ../docker/Dockerfile

FROM python:3.8

# As of 2021-11-03, SageMaker only supports 1000/100 or 0/0 as the possible UID/GID values.
ARG NB_USER="sagemaker-user"
ARG NB_UID="1000"
ARG NB_GID="100"

# Setup the "sagemaker-user" user with root privileges.
RUN \
    apt-get update && \
    apt-get install -y sudo && \
    useradd -m -s /bin/bash -N -u $NB_UID $NB_USER && \
    chmod g+w /etc/passwd && \
    echo "${NB_USER}    ALL=(ALL)    NOPASSWD:    ALL" >> /etc/sudoers && \
    # Prevent apt-get cache from being persisted to this layer.
    rm -rf /var/lib/apt/lists/*
    
# Make the default shell bash (vs "sh") for a better Jupyter terminal UX
ENV SHELL=/bin/bash
ENV PATH="/home/sagemaker-user/.local/bin:${PATH}"

RUN pip install ipykernel && \
        python -m ipykernel install --sys-prefix

USER $NB_UID


At high-level the Dockerfile specifies the following operations for building this container:
<ul>
    <li>Start from Python 3.8 image</li>
    <li>Define some variables to be used at build time to give root privileges to sagemaker-user</li>
    <li>Make bash the default shell</li>
    <li>Install the IPython kernel.</li>
    <li>Set the user to use when running the image.</li>
</ul>

### Build and push the container
We are now ready to build this container and push it to Amazon ECR. This task is executed using a shell script stored in the ../script/ folder. Let's take a look at this script and then execute it.

In [107]:
! pygmentize ../scripts/build_and_push.sh

ACCOUNT_ID=$1
REGION=$2
REPO_NAME=$3

docker build -f ../docker/Dockerfile -t $REPO_NAME ../docker

docker tag $REPO_NAME $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/$REPO_NAME:latest

$(aws ecr get-login --no-include-email --registry-ids $ACCOUNT_ID)

aws ecr describe-repositories --repository-names $REPO_NAME || aws ecr create-repository --repository-name $REPO_NAME

docker push $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/$REPO_NAME:latest


<h3>--------------------------------------------------------------------------------------------------------------------</h3>

The script builds the Docker container, then creates the repository if it does not exist, and finally pushes the container to the ECR repository. The build task requires a few minutes to be executed the first time, then Docker caches build outputs to be reused for the subsequent build operations.

In [108]:
! ../scripts/build_and_push.sh $account_id $region $ecr_repository_name

Sending build context to Docker daemon  4.608kB
Step 1/9 : FROM python:3.8
 ---> 53da5c105f01
Step 2/9 : ARG NB_USER="sagemaker-user"
 ---> Using cache
 ---> 1f094f811114
Step 3/9 : ARG NB_UID="1000"
 ---> Using cache
 ---> af0329a58f08
Step 4/9 : ARG NB_GID="100"
 ---> Using cache
 ---> 332a08c13795
Step 5/9 : RUN     apt-get update &&     apt-get install -y sudo &&     useradd -m -s /bin/bash -N -u $NB_UID $NB_USER &&     chmod g+w /etc/passwd &&     echo "${NB_USER}    ALL=(ALL)    NOPASSWD:    ALL" >> /etc/sudoers &&     rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 2cb7da73b878
Step 6/9 : ENV SHELL=/bin/bash
 ---> Using cache
 ---> f576bbc3bd89
Step 7/9 : ENV PATH="/home/sagemaker-user/.local/bin:${PATH}"
 ---> Using cache
 ---> a2cefae7c23e
Step 8/9 : RUN pip install ipykernel &&         python -m ipykernel install --sys-prefix
 ---> Using cache
 ---> 7d8e829cf451
Step 9/9 : USER $NB_UID
 ---> Using cache
 ---> 25920b912845
Successfully built 25920b912845
Successfully tagge

## Validate the container locally

In [109]:
local_container_image = '{0}:latest'.format(ecr_repository_name)
print(local_container_image)

sagemaker-studio-containers/basic-studio-container:latest


Let's check if the container defines kernels appropriately:

In [110]:
!docker run $local_container_image bash -c 'jupyter-kernelspec list'

Available kernels:
  python3    /usr/local/share/jupyter/kernels/python3


Now we can run the container with a KernelGateway, to validate if the kernels are visible from the exposed REST endpoint.

In [111]:
container_id = !docker run -d -p 8888:8888 $local_container_image bash -c 'pip install jupyter_kernel_gateway  && jupyter kernelgateway --ip 0.0.0.0 --debug --port 8888' 
container_id = container_id[0]
print(container_id)

c1b5e1554893395bcbcdcd6fe37e53f8d3a2e202f56af8dd83c7d79eeb6be39a


Let's wait 30 seconds to allow the KernelGateway to start and then we can try invoking the API returning kernelspec.

In [ ]:
import time
time.sleep(30)

In [113]:
kernelspecs = !curl http://0.0.0.0:8888/api/kernelspecs
print(kernelspecs[5])

{"default": "python3", "kernelspecs": {"python3": {"name": "python3", "spec": {"argv": ["/usr/local/bin/python", "-m", "ipykernel_launcher", "-f", "{connection_file}"], "env": {}, "display_name": "Python 3", "language": "python", "interrupt_mode": "signal", "metadata": {}}, "resources": {"logo-64x64": "/kernelspecs/python3/logo-64x64.png", "logo-32x32": "/kernelspecs/python3/logo-32x32.png"}}}}


Let's display this JSON nicely.

In [114]:
from IPython.display import JSON
JSON(kernelspecs[5], expanded=True)

<IPython.core.display.JSON object>

As expected the python3 kernel specification is returned. Now we can kill the local container.

In [115]:
! docker kill $container_id

c1b5e1554893395bcbcdcd6fe37e53f8d3a2e202f56af8dd83c7d79eeb6be39a


## Create SageMaker Image

Once the container is ready, we need to:
 - create a SageMaker Image referencing the container image pushed to ECR
 - create an App Image Configuration for running the SageMaker Image as a KernelGateway app
 - associate the SageMaker Image and its App Image Configuration to our Amazon SageMaker Studio domain

In [122]:
container_image_uri = '{0}.dkr.ecr.{1}.amazonaws.com/{2}:latest'.format(account_id, region, ecr_repository_name)
print(container_image_uri)

825935527263.dkr.ecr.eu-west-1.amazonaws.com/sagemaker-studio-containers/basic-studio-container:latest


In [123]:
client = boto3.client('sagemaker')

image_response = client.create_image(
    Description='Python 3.8 kernel image',
    DisplayName='Python 3.8',
    ImageName='python38-custom-image',
    RoleArn=role
)
print(image_response)

{'ImageArn': 'arn:aws:sagemaker:eu-west-1:825935527263:image/python38-custom-image', 'ResponseMetadata': {'RequestId': '884bbc59-296f-42ee-9d2d-543c513a1276', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '884bbc59-296f-42ee-9d2d-543c513a1276', 'content-type': 'application/x-amz-json-1.1', 'content-length': '83', 'date': 'Wed, 24 Mar 2021 18:38:40 GMT'}, 'RetryAttempts': 0}}


In [124]:
image_version_response = client.create_image_version(
    BaseImage=container_image_uri,
    ImageName='python38-custom-image'
)
print(image_version_response)

{'ImageVersionArn': 'arn:aws:sagemaker:eu-west-1:825935527263:image-version/python38-custom-image/1', 'ResponseMetadata': {'RequestId': '656b971f-819d-407d-9766-b0c0835ec4f6', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '656b971f-819d-407d-9766-b0c0835ec4f6', 'content-type': 'application/x-amz-json-1.1', 'content-length': '100', 'date': 'Wed, 24 Mar 2021 18:38:41 GMT'}, 'RetryAttempts': 0}}


Let's define the Kernel Gateway App image configuration.

Notes:
- <strong>Name and DisplayName must match exactly the names in the kernelspec exposed by the container (see previous section)</strong>
- <strong>DefaultUid and DefaultGid must match exactly the UID and GID defined in the container</strong>


In [125]:
app_image_config_response = client.create_app_image_config(
    AppImageConfigName='python38-custom-app-image-config',
    KernelGatewayImageConfig={
        'KernelSpecs': [
            {
                'Name': 'python3',
                'DisplayName': 'Python 3'
            },
        ],
        'FileSystemConfig': {
            'MountPath': '/home/sagemaker-user',
            'DefaultUid': 1000,
            'DefaultGid': 100
        }
    }
)
print(app_image_config_response)

{'AppImageConfigArn': 'arn:aws:sagemaker:eu-west-1:825935527263:app-image-config/python38-custom-app-image-config', 'ResponseMetadata': {'RequestId': 'af776971-a273-4aff-9421-d245b73fef0d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'af776971-a273-4aff-9421-d245b73fef0d', 'content-type': 'application/x-amz-json-1.1', 'content-length': '114', 'date': 'Wed, 24 Mar 2021 18:38:56 GMT'}, 'RetryAttempts': 0}}


## Testing the custom image and kernel

To test the custom image and the exposed Python 3.8 kernel, we are going to update an existing Amazon SageMaker Studio domain. If you don't have a Studio domain ready, please follow the instructions at: https://docs.aws.amazon.com/sagemaker/latest/dg/gs.html

In [126]:
domain_id = 'd-szhayk8bljvj'

update_domain_response = client.update_domain(
    DomainId=domain_id,
    DefaultUserSettings={
        'KernelGatewayAppSettings': {
            'CustomImages': [
                {
                    'ImageName': 'python38-custom-image',
                    'ImageVersionNumber': 1,
                    'AppImageConfigName': 'python38-custom-app-image-config'
                },
            ]
        }
    }
)
print(update_domain_response)

{'DomainArn': 'arn:aws:sagemaker:eu-west-1:825935527263:domain/d-szhayk8bljvj', 'ResponseMetadata': {'RequestId': 'ecb61392-bb0d-4421-9af9-c1de3b759a7f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'ecb61392-bb0d-4421-9af9-c1de3b759a7f', 'content-type': 'application/x-amz-json-1.1', 'content-length': '78', 'date': 'Wed, 24 Mar 2021 18:39:08 GMT'}, 'RetryAttempts': 0}}


Now we need to move to the SageMaker Studio UI to check if the custom image and kernel work as expected.

In [121]:
domain_id = 'd-szhayk8bljvj'

update_domain_response = client.update_domain(
    DomainId=domain_id,
    DefaultUserSettings={
        'KernelGatewayAppSettings': {
            'CustomImages': [
            ]
        }
    }
)
print(update_domain_response)


client.delete_image_version(
    ImageName='python38-custom-image',
    Version=1
)
client.delete_image(
    ImageName='python38-custom-image'
)
response = client.delete_app_image_config(
    AppImageConfigName='python38-custom-app-image-config'
)

{'DomainArn': 'arn:aws:sagemaker:eu-west-1:825935527263:domain/d-szhayk8bljvj', 'ResponseMetadata': {'RequestId': '82d23834-32b1-4d13-a620-88a13ef7b8a2', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '82d23834-32b1-4d13-a620-88a13ef7b8a2', 'content-type': 'application/x-amz-json-1.1', 'content-length': '78', 'date': 'Wed, 24 Mar 2021 18:37:13 GMT'}, 'RetryAttempts': 0}}
